# Combinators

Combinators:

* are functions that construct functions from other functions.
* provides a powerful mechanism for reusing logic...
  without having to anticpate the future.

In [140]:

from typing import Any, Optional, Union, List, Tuple, Dict, Mapping, Callable, Type, Literal
from numbers import Number
from collections.abc import Collection, Sequence
from dataclasses import dataclass, field
import re
import sys
import logging
from pprint import pformat
import dis
from icecream import ic

logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)

# https://stackoverflow.com/a/47024809/1141958
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [141]:
def identity(x: Any) -> Any:
  'Returns the first argument.'
  return x

identity(2)
f = identity
f
f(3)

2

<function __main__.identity(x: Any) -> Any>

3

In [142]:
def plus_three(x):
  return x + 3

g = plus_three  # g is a function
g
g(2)

<function __main__.plus_three(x)>

5

# Second-Order Functions

Second-Order Functions return other functions.

In [143]:
# Functions with zero or more arguments that return anything.
Variadic = Callable[..., Any]

def constantly(x: Any) -> Variadic:
  'Returns a function that returns a constant value.'
  return lambda *_args, **_kwargs: x

always_7 = constantly(7)
always_7

<function __main__.constantly.<locals>.<lambda>(*_args, **_kwargs)>

In [144]:
always_7()
always_7(11)
always_7(13, 17)

7

7

7

# Object Adapters

In [145]:
# Function with one argument that returns anything.
Unary = Callable[[Any], Any]

# A value `x` that supports `x[i]`:
Indexable = Union[List, Tuple, Dict]

def at(i: Any) -> Unary:
  'Returns a function `f(x)` that returns `x[i]`.'
  return lambda x: x[i]

f = at(2)
f([0, 1, 2, 3])

g = at("a")
g({"a": 1, "b": 2})

2

1

In [146]:
def indexed(x: Indexable) -> Unary:
  'Returns a function `f(i)` that returns `x[i]`.'
  return lambda i: x[i]

f = indexed([0, 1, 2, 3])
f(2)

g = indexed({"a": 1, "b": 2})
g("a")

2

1

# Object Accessors

In [147]:
@dataclass
class Position:
  x: int = 0
  y: int = 0

p = Position(2, 3)
p

def getter(name: str) -> Unary:
  return lambda obj: getattr(obj, name)

def setter(name: str) -> Unary:
  return lambda obj, val: setattr(obj, name, val)

def accessor(name: str) -> Callable:
  def g(obj, *val):
    if val:
      return setattr(obj, name, val[0])
    return getattr(obj, name)
  return g

Position(x=2, y=3)

In [148]:
g = getter('x')
g(p)

2

In [149]:
s = setter('x')
s(p, 5)
p

Position(x=5, y=3)

In [150]:

a = accessor('x')
a(p)
a(p, 7)
p

5

Position(x=7, y=3)

# Object as Dict

In [151]:
def object_get(obj):
  def g(name, *default):
    if default and not hasattr(obj, name):
      return default[0]
    return getattr(obj, name)
  return g

p = Position(2, 3)
f = object_get(p)
f('x')
f('y')

2

3

In [152]:
try:
  f('z')
except AttributeError as e:
  print(repr(e))

AttributeError("'Position' object has no attribute 'z'")


In [153]:
f('z', 13)

13

# Generators

In [154]:
def counter(i: int = 0):
  'Returns a "generator" function that returns `i`, `i + 1`, `i + 2`, ... .'
  i -= 1
  def g():
    nonlocal i
    return (i := i + 1)
  return g

c = counter(2)
c
c()
c()
c()

<function __main__.counter.<locals>.g()>

2

3

4

# Stateful Combinators

In [155]:
def with_index(f: Callable, i: int = 0):
  c = counter(i)
  return lambda *args, **kwargs: f(c(), *args, **kwargs)

def conjoin(a, b):
  return (a, b)

dict(map(with_index(conjoin), ["a", "b", "c", "d"]))

{0: 'a', 1: 'b', 2: 'c', 3: 'd'}

# Predicates

In [156]:

# Functions with zero or more arguments that return a boolean.
Predicate = Callable[..., bool]

def is_string(x: Any) -> bool:
  'Returns true if `x` is a string.'
  return isinstance(x, str)

is_string("hello")
is_string(3)

True

False

In [157]:

# Functions with zero or more arguments that return a boolean.
Predicate = Callable[..., bool]

def is_string(x: Any) -> bool:
  'Returns true if `x` is a string.'
  return isinstance(x, str)

is_string("hello")
is_string(3)

True

False

# Predicate Combinators

In [158]:
# Functions that take a Predicate and return a new Predicate.
PredicateCombinator = Callable[[Predicate], Predicate]

def not_(f: Predicate) -> Predicate:
  'Returns a function that logically negates the result of the given function.'
  return lambda *args, **kwargs: not f(*args, **kwargs)

f = not_(is_string)
f("hello")
f(3)

False

True

# Mapping functions over sequences

In [159]:
def map(f: Unary, xs: Sequence) -> Sequence:
  'Returns a sequence of `f(x)` for each element `x` in `xs`.'
  return [f(x) for x in xs]

items = [1, "string", False, True, None]
map(identity, items)
map(always_7, items)
map(is_string, items)
map(not_(is_string), items)
map(plus_three, [3, 5, 7, 11])

[1, 'string', False, True, None]

[7, 7, 7, 7, 7]

[False, True, False, False, False]

[True, False, True, True, True]

[6, 8, 10, 14]

# Filtering Sequences with Predicates

In [160]:
def filter(f: Unary, xs: Sequence) -> Sequence:
  'Returns a sequence of the elements of `xs` for which `f` returns true.'
  return [x for x in xs if f(x)]

items = [1, "string", False, True, None]
filter(is_string, items)
filter(not_(is_string), items)

['string']

[1, False, True, None]

# Reducing Sequences with Binary Functions

In [161]:
# Functions with two arguments that return anything.
Binary = Callable[[Any, Any], Any]

def reduce(f: Binary, init: Any, xs: Sequence) -> Sequence:
  'Returns the result of `init = f(x, init)` for each element `x` in `xs`.'
  for x in xs:
    init = f(init, x)
  return init

def add(x, y):
  return x + y

reduce(add, 2, [3, 5, 7])
a_list_of_strings = ["A", "List", 'Of', 'Strings']
reduce(add, "Here Is ", a_list_of_strings)

17

'Here Is AListOfStrings'

In [162]:
def conjoin(x, y):
  return (x, y)

items = [3, "a", 5, "b", 7, "c", 11, True]
reduce(conjoin, 2, items)

((((((((2, 3), 'a'), 5), 'b'), 7), 'c'), 11), True)

In [163]:
# Concat all strings:
reduce(add, "", filter(is_string, items))

# Sum of all numbers:
def is_number(x: Any) -> bool:
  return not isinstance(x, bool) and isinstance(x, Number)
reduce(add, 0, filter(is_number, items))

# Sum all non-strings:
reduce(add, 0, filter(not_(is_string), items))

'abc'

26

27

# Map as a Reduction

In [164]:
def map_r(f: Unary, xs: Sequence) -> Sequence:
  def acc(seq, x):
    return seq + [f(x)]
  return reduce(acc, [], xs)

map(plus_three, [3, 5, 7, 11])
map_r(plus_three, [3, 5, 7, 11])

[6, 8, 10, 14]

[6, 8, 10, 14]

# Filter as a Reduction

In [165]:
def filter_r(f: Unary, xs: Sequence) -> Sequence:
  def acc(seq, x):
    return seq + [x] if f(x) else seq
  return reduce(acc, [], xs)

items
filter(is_string, items)
filter_r(is_string, items)

[3, 'a', 5, 'b', 7, 'c', 11, True]

['a', 'b', 'c']

['a', 'b', 'c']

# Function Composition

In [166]:
def compose(*callables) -> Variadic:
  'Returns the composition one or more functions, in reverse order.'
  'For example, `compose(g, f)(x, y)` is equivalent to `g(f(x, y))`.'
  f: Callable = callables[-1]
  gs: Sequence[Unary] = tuple(reversed(callables[:-1]))
  def h(*args, **kwargs):
    result = f(*args, **kwargs)
    for g in gs:
      result = g(result)
    return result
  return h

def multiply_by_3(x):
  return x * 3

plus_three(multiply_by_3(5))

f = compose(plus_three, multiply_by_3)
f(5)

18

18

# Interlude

In [167]:
def modulo(modulus: int) -> Callable[[int], int]:
  return lambda x: x % modulus

mod_3 = modulo(3)
map(mod_3, range(10))

[0, 1, 2, 0, 1, 2, 0, 1, 2, 0]

In [168]:
f = compose(indexed(a_list_of_strings), mod_3)
map(f, range(10))

['A', 'List', 'Of', 'A', 'List', 'Of', 'A', 'List', 'Of', 'A']

# Partial Application

In [169]:

def add_and_multiply(a, b, c):
  return (a + b) * c

add_and_multiply(2, 3, 5)

def partial(f: Callable, *args, **kwargs) -> Callable:
  'Returns a function that prepends `args` and merges `kwargs`.'
  def g(*args2, **kwargs2):
    return f(*(args + args2), **dict(kwargs, **kwargs2))
  return g

f = partial(add_and_multiply, 2)
f(3, 5)

25

25

# Methods are Partially Applied Functions

In [170]:
a = 2
b = 3
a + b
a.__add__(b)    # eqv. to `a + b`
f = a.__add__
f
f(7)

5

5

<method-wrapper '__add__' of int object at 0x100c0f4a0>

9

# Debugging

In [171]:
def trace(
    name: str,
    log: Unary,
    f: Callable,
  ) -> Callable:
  def g(*args, **kwargs):
    msg = f"{name}({format_args(args, kwargs)})"
    log(f"{msg} => ...")
    result = f(*args, **kwargs)
    log(f"{msg} => {result!r}")
    return result
  return g

def format_args(args, kwargs):
  return ', '.join(map(repr, args) + [f'{k}={v!r}' for k, v in kwargs])

def log(msg):
  sys.stderr.write(f'  ## {msg}\n')

f = compose(str, plus_three)
map(f, [2, 3, 5])

g = trace("g", log, f, )
map(g, [2, 3, 5])

map_g = trace("map_g", log, partial(map, g))
map_g([2, 3, 5])


['5', '6', '8']

  ## g(2) => ...
  ## g(2) => '5'
  ## g(3) => ...
  ## g(3) => '6'
  ## g(5) => ...
  ## g(5) => '8'


['5', '6', '8']

  ## map_g([2, 3, 5]) => ...
  ## g(2) => ...
  ## g(2) => '5'
  ## g(3) => ...
  ## g(3) => '6'
  ## g(5) => ...
  ## g(5) => '8'
  ## map_g([2, 3, 5]) => ['5', '6', '8']


['5', '6', '8']

# Error Handlers

In [172]:
def except_(f: Variadic, ex_class, error: Unary) -> Callable:
  def g(*args, **kwargs):
    try:
      return f(*args, **kwargs)
    except ex_class as exc:
      return error((exc, args, kwargs))
  return g

f = except_(plus_three, TypeError, compose(partial(logging.error, 'plus_three: %s'), repr))
f(2)
f('Nope')

5

ERROR:root:plus_three: (TypeError('can only concatenate str (not "int") to str'), ('Nope',), {})


# Arity Reduction

In [173]:
def unary(f: Variadic) -> Unary:
  return lambda *args, **kwargs: f((args, kwargs))

f = unary(identity)
f()
f(1)
f(1, 2)
f(a=1, b=2)

((), {})

((1,), {})

((1, 2), {})

((), {'a': 1, 'b': 2})

# Predicators

In [174]:
def re_pred(pat: str, re_func: Callable = re.search) -> Predicate:
  'Returns a predicate that matches a regular expression.'
  rx = re.compile(pat)
  return lambda x: re_func(rx, str(x)) is not None

re_pred("ab")("abc")
re_pred("ab")("nope")

True

False

In [175]:
def default(f: Variadic, g: Variadic) -> Variadic:
  def h(*args, **kwargs):
    if (result := f(*args, **kwargs)) is not None:
      return result
    return g(*args, **kwargs)
  return h

asdf

# Logical Predicate Composers

In [176]:
def and_(f: Variadic, g: Variadic) -> Variadic:
  'Returns a function `h(x, ...)` that returns `f(x, ...) and g(x, ...).'
  return lambda *args, **kwargs: f(*args, **kwargs) and g(*args, **kwargs)

def or_(f: Variadic, g: Variadic) -> Variadic:
  'Returns a function `h(x, ...)` that returns `f(x, ...) or g(x, ...).'
  return lambda *args, **kwargs: f(*args, **kwargs) or g(*args, **kwargs)

def is_string(x):
  return isinstance(x, str)

is_word = and_(is_string, re_pred(r'^[a-z]+$'))
is_word("hello")
is_word("not-a-word")
is_word(2)
is_word(None)

True

False

False

False

In [177]:
# If x is a number add three:
f = and_(is_number, plus_three)
# If x is a string, is it a word?:
g = and_(is_string, is_word)
# One or the other:
h = or_(f, g)
items = ["hello", "not-a-word", 2, None]
map(g, items)

[True, False, False, False]

In [178]:
Procedure = Callable[[], Any]

def if_(f: Variadic, g: Unary, h: Unary) -> Variadic:
  def i(*args, **kwargs):
    if (result := f(*args, **kwargs)):
      return g()
    return h()
  return i

# Operator Predicates

In [179]:

def binary_op(op: str) -> Optional[Callable[[Any, Any], Any]]:
  'Returns a function for a binary operator by name.'
  return BINARY_OPS.get(op)

BINARY_OPS = {
  '==': lambda a, b: a == b,
  '!=': lambda a, b: a != b,
  '<':  lambda a, b: a < b,
  '>':  lambda a, b: a > b,
  '<=': lambda a, b: a <= b,
  '>=': lambda a, b: a >= b,
  'and': lambda a, b: a and b,
  'or': lambda a, b: a or b,
}

binary_op('==') (2, 2)
binary_op('!=') (2, 2)

True

False

In [180]:
# Create a table where `x OP y` is true:
[
  f'{x} {op} {y}'
  for op in ['<', '==', '>']
  for x in (2, 3, 5)
  for y in (2, 3, 5)
  if binary_op(op)(x, y)
]

['2 < 3',
 '2 < 5',
 '3 < 5',
 '2 == 2',
 '3 == 3',
 '5 == 5',
 '3 > 2',
 '5 > 2',
 '5 > 3']

In [181]:
def op_pred(op: str, b: Any) -> Optional[Predicate]:
  'Returns a predicate function given an operator name and a constant.'
  if pred := binary_op(op):
    return lambda a: pred(a, b)
  if op == "not":
    return lambda a: not a
  if op == "~=":
    return re_pred(b)
  if op == "~!":
    return not_(re_pred(b))
  return None

In [182]:
f = op_pred(">", 3)
f(2)
f(5)

False

True

In [183]:
g = op_pred("~=", 'ab+c')
g('')
g('ab')
g('abbbcc')

False

False

True

# Sequencing

In [184]:
def progn(*fs: Sequence[Callable]) -> Callable:
  'Returns a function that calls each function in turn and returns the last result.'
  def g(*args, **kwargs):
    result = None
    for f in fs:
      result = f(*args, **kwargs)
    return result
  return g

In [185]:
def prog1(*fs: Sequence[Callable]) -> Callable:
  'Returns a function that calls each function in turn and returns the last result.'
  def g(*args, **kwargs):
    result = fs[0](*args, **kwargs)
    for f in fs[1:]:
      result = f(*args, **kwargs)
    return result
  return g

In [186]:
##############################################
## Extraction
##############################################

def reverse_apply(x: Any) -> Callable:
  return lambda f, *args, **kwargs: f(x, *args, **kwargs)

reverse_apply(1) (plus_three)

4

In [187]:
def demux(*funcs) -> Unary:
  'Return a function `h(x)` that returns `[f(x), g(x), ...].'
  return lambda x: map(reverse_apply(x), funcs)

demux(identity, len, compose(tuple, reversed))("abcd")

['abcd', 4, ('d', 'c', 'b', 'a')]

# Parser Combinators

In [188]:
# Parser input: a sequence of lexemes:
Input = Sequence[Any]

# A parsed value and remaining input:
Parsed = Tuple[Any, Input]

# A parser matches the input sequence and produces a result or nothing:
Parser = Callable[[Input], Optional[Parsed]]

In [189]:

def show_match(p: Parser) -> Variadic:
  def g(input: Input):
    return (p(input) or False, '<=', input)
  return g

In [190]:
first = at(0)
def rest(x: Input) -> Input:
  return x[1:]

def equals(x) -> Parser:
  'Returns a parser that matches `x`.'
  def g(input: input):
    y = first(input)
    logging.debug("equals(%s, %s)", repr(x), repr(y))
    if x == y:
      return (y, rest(input))
  return g

f = equals('a')
f(['a'])
f(['b', 2])


DEBUG:root:equals('a', 'a')


('a', [])

DEBUG:root:equals('a', 'b')


In [191]:
def which(p: Predicate) -> Parser:
  'Returns a parser for the next lexeme when `p(lexeme)` is true.'
  def g(input: Input):
    if p(first(input)):
      return (first(input), rest(input))
  return g

g = which(is_string)
g(['a'])
g([2])
g(['a', 'b'])

def alternation(*ps) -> Parser:
  def g(input):
    for p in ps:
      if (result := p(input)):
        return result
  return g

g = alternation(which(is_string), which(is_number))
g(['a'])
g([2])
g([False])


('a', [])

('a', ['b'])

('a', [])

(2, [])

# Sequence Parsers

In [192]:
ParsedSequence = Tuple[Sequence, Input]
SequenceParser = Callable[[Input], Optional[ParsedSequence]]

def one(p: Parser) -> SequenceParser:
  'Returns a parser for one lexeme.'
  def g(input: Input):
    if input and (result := p(input)):
      parsed, input = result
      return ([parsed], input)
  return g

g = one(which(is_string))
g([])
g(['a'])
g([2])
g(['a', 'b'])


(['a'], [])

(['a'], ['b'])

In [193]:
def zero_or_more(p: Parser) -> SequenceParser:
  'Returns a parser for zero or more lexemes.'
  def g(input: Input):
    acc = []
    while input and (result := p(input)):
      parsed, input = result
      acc.append(parsed)
    return (acc, input)
  return g

g = zero_or_more(which(is_string))
g([])
g(['a'])
g([2])
g(['a', 'b'])
g(['a', 'b', 2])
g(['a', 'b', 3, 5])

([], [])

(['a'], [])

([], [2])

(['a', 'b'], [])

(['a', 'b'], [2])

(['a', 'b'], [3, 5])

In [194]:
def one_or_more(p: Parser) -> SequenceParser:
  'Returns a parser for one or more lexemes as a sequence.'
  p = zero_or_more(p)
  def g(input: Input):
    if (result := p(input)) and len(result[0]) >= 1:
      return result
  return g

In [195]:
g = one_or_more(which(is_string))
g([])
g(['a'])
g([2])
g(['a', 'b'])
g(['a', 'b', 2])
g(['a', 'b', 3, 5])

(['a'], [])

(['a', 'b'], [])

(['a', 'b'], [2])

(['a', 'b'], [3, 5])

In [196]:
def sequence_of(*parsers) -> SequenceParser:
  'Returns a parser for parsers of a sequence.'
  def g(input: Input):
    acc = []
    for p in parsers:
      if result := p(input):
        parsed, input = result
        acc.extend(parsed)
      else:
        return None
    return (acc, input)
  return g

g = sequence_of(one(which(is_string)), one(which(is_string)))
g([])
g(['a'])
g([2])
g(['a', 'b'])
g(['a', 'b', 2])
g(['a', 'b', 3, 5])

(['a', 'b'], [])

(['a', 'b'], [2])

(['a', 'b'], [3, 5])

In [197]:
g = sequence_of(one_or_more(which(is_number)))
g([])
g(['a'])
g([2])
g([2, 3])
g([2, 3, False])

([2], [])

([2, 3], [])

([2, 3], [False])

In [198]:
g = sequence_of(one(which(is_string)), one_or_more(which(is_number)))
g([])
g(['a'])
g([2])
g(['a', 'b'])
g(['a', 2])
g(['a', 2, 3])
g(['a', 2, 'b', 3])
g(['a', 2, 3, False])
g(['a', 2, 3, False, 'more'])

(['a', 2], [])

(['a', 2, 3], [])

(['a', 2], ['b', 3])

(['a', 2, 3], [False])

(['a', 2, 3], [False, 'more'])

# Parser Grammar

In [199]:
def take_while(f: Unary) -> Unary:
  def g(seq):
    acc = []
    while seq and (x := f(seq[0])):
      acc.append(x)
    return acc
  return g

In [209]:
def hash_deep(obj):
  def mix(a, b):
    a = a - 1
    if a < 0:
      a = - a
    b = b - 1
    if b < 0:
      b = - b
    return a ^ b
  def hash_deep_seq(obj):
    h = len(obj).__hash__()
    for e in obj:
      h = mix(hash_deep(e), h)
    return h
  if f := obj.__hash__:
    return f()
  h = obj.__class__.__name__.__hash__()
  if isinstance(obj, (list, tuple)):
    return mix(h, hash_deep_seq(obj))
  if isinstance(obj, (dict)):
    return mix(h, hash_deep_seq(obj))
  return -1

hash_deep(123)
hash_deep(123.45)
hash_deep(True)
hash_deep(False)
hash_deep([])
hash_deep('a')
hash_deep(['a'])
hash_deep((1))
hash_deep({})
hash_deep({"a": 1})

def dict_deep():
  d = {}

  def d_get(k):
    h = hash_deep(k)
    if bucket := d.get(h):
      for e in bucket:
        if e[0] == k:
          return e[1]
    return None

  def d_set(k, v):
    h = hash_deep(k)
    if not (bucket := d.get(h)):
      d[h] = bucket = []
    for e in bucket:
      if e[0] == k:
        e[1] = v
        return
    bucket.append([k, v])

  def g(k, *v):
    if v:
      d_set(k, v[0])
    else:
      return d_get(k)
  return g


123

1037629354146168955

1

0

3959704358772763333

89209573381734311

4021535800558359905

1

5882399599101153001

5809244287076430157

In [215]:
def eat(rx: str):
  p = re.compile(rx)
  return lambda s: re.sub(p, '', s)

def lexeme(pat: str, post = at(0)):
  post = post or (lambda m: m[0])
  rx = re.compile(pat)
  ws = eat(r'^\s+')
  def g(input):
    input = ws(input)
    if m := re.match(rx, input):
      return (post(m), input[len(m[0]):])
  return g

def cache(p):
  d = {}
  def g(input):
    if v := d.get(input):
      logging.debug('cached : %s => %s', repr(input), repr(v[0]))
      return v[0]
    v = p(input)
    d[input] = (v,)
    return v
  return g


In [217]:
def grammar_parser():
  env = {}
  def _(id):
    # nonlocal env
    # logging.debug('_(%s)', repr(id))
    return lambda *args: env[id](*args)

  def action(p: Parser, action: Unary) -> Parser:
    def g(input: Input):
      if result := p(input):
        value = action(result[0])
        return (value, result[1])
    return g

  def definition(id, p, act=None):
    log = partial(logging.debug, '  # %s')
    # p = trace(id, log, p)
    p = cache(p)
    a = None
    if act is True:
      a = lambda x: (id, x)
    elif isinstance(act, str):
      a = lambda x: (act, x)
    elif act:
      a = act
    if a:
      p = action(p, a)
    env[id] = p

  definition('int',     lexeme(r'[-+]?\d+',               lambda m: ('int', int(m[0]))))
  definition('str'   ,  lexeme(r'"(\\"|\\?[^"]+)*"',      lambda m: ('str', eval(m[0]))))
  definition('re',      lexeme(r'/((\\/|(\\?[^/]+))*)/',  lambda m: ('re', re.compile(m[1]))))
  definition('name',    lexeme(r'[a-zA-Z][a-zA-Z0-9_]*',  lambda m: ('name', m[0])))
  definition('terminal', alternation(_("str"), _("re"), _('int')))
  definition('non-terminal', _('name'))
  definition('pattern', alternation(_('non-terminal'), _('terminal')))
  definition('sequence_of',
    sequence_of(one_or_more(_('pattern'))))
  definition('alternation',
    sequence_of(one(_('sequence_of')), one(lexeme(r'\|')), one(_('production'))),
    lambda x: ('alternation', x[0], x[2]))
  definition('production',
    alternation(_('alternation'), _('sequence_of')))
  definition('definition',
    sequence_of(one(_('name')), one(lexeme(r"=")), one(_('production')), one(lexeme(r';'))),
    lambda x: ('definition', x[0], x[2]))
  definition('grammar',
    sequence_of(one_or_more(_('definition'))))

  def g(input, start='grammar'):
    return env[start](input)
  return g

gram = grammar_parser()

In [218]:
def test(input, name='definition'):
  logging.debug("============================================\n")
  ic(input)
  result = gram(input, name)
  ic(name)
  ic(input)
  ic(result)

test('"asdf"', 'pattern')
test('  a = b c;')
test('a =b c|d;')
test('a = b c | d | e f;')
test('a = "foo";')

DEBUG:root:============================================

ic| input: '"asdf"'
ic| name: 'pattern'
ic| input: '"asdf"'
ic| result: (('str', 'asdf'), '')
DEBUG:root:============================================

ic| input: '  a = b c;'
DEBUG:root:cached : ' b c;' => ([('name', 'b'), ('name', 'c')], ';')
ic| name: 'definition'
ic| input: '  a = b c;'
ic| result: (('definition', ('name', 'a'), [('name', 'b'), ('name', 'c')]), '')
DEBUG:root:============================================

ic| input: 'a =b c|d;'
DEBUG:root:cached : ';' => None
DEBUG:root:cached : 'd;' => ([('name', 'd')], ';')
ic| name: 'definition'
ic| input: 'a =b c|d;'
ic| result: (('definition',
              ('name', 'a'),
              ('alternation', [('name', 'b'), ('name', 'c')], [('name', 'd')])),
             '')
DEBUG:root:============================================

ic| input: 'a = b c | d | e f;'
DEBUG:root:cached : ';' => None
DEBUG:root:cached : ' e f;' => ([('name', 'e'), ('name', 'f')], ';')
ic| name: 'definit

In [ ]:
#################################################
## Other
#################################################

def projection(key: Any, default: Any = None) -> Callable:
  'Returns a function `f(a)` that returns `a.get(key, default)`.'
  return lambda a: a.get(key, default)


# Mapcat (aka Flat-Map)

In [ ]:

ConcatableUnary = Callable[[Any], Sequence]

def mapcat(f: ConcatableUnary, xs: Sequence):
  'Concatenate the results of `map(f, xs)`.'
  return reduce(add, [], map(f, xs))

def duplicate(n, x):
  return [x] * n

duplicate_each_3_times = partial(mapcat, partial(duplicate, 3))
duplicate_each_3_times([".", "*"])
duplicate_each_3_times(range(4, 7))


['.', '.', '.', '*', '*', '*']

[4, 4, 4, 5, 5, 5, 6, 6, 6]

# Manipulating Arguments

In [ ]:
def reverse_args(f: Callable) -> Callable:
  def g(*args, **kwargs):
    return f(*reversed(args), **kwargs)
  return g

def divide(x, y):
  return x / y

divide(2, 3)
reverse_args(divide)(2, 3)

reduce(reverse_args(add), " reversed ", a_list_of_strings)
reduce(reverse_args(conjoin), 2, [3, 5, 7])

0.6666666666666666

1.5

'StringsOfListA reversed '

(7, (5, (3, 2)))